In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## **Load Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = "/content/drive/My Drive/dataset-fixed"
train_dir = f"{data_dir}/train"
test_dir = f"{data_dir}/test"

## **Data Preparation**

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## **Load Pre-Trained VGG16**

In [ ]:
model = models.vgg16(pretrained=True)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, len(train_dataset.classes))
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 74.2MB/s]


## **Loss function and optimizer**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## **Training Model**

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()
                loader = dataloaders['train']
            else:
                model.eval()
                loader = dataloaders['test']

            running_loss = 0.0
            running_corrects = 0

            with tqdm(loader, desc=f"{phase.capitalize()} Progress", leave=False) as progress:
                for inputs, labels in progress:
                    inputs, labels = inputs.to(device), labels.to(device)

                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    progress.set_postfix(
                        loss=f"{running_loss / ((progress.n + 1) * inputs.size(0)):.4f}",
                        acc=f"{(running_corrects.double() / (progress.n + 1)):.4f}"
                    )

            epoch_loss = running_loss / len(loader.dataset)
            epoch_acc = running_corrects.double() / len(loader.dataset)
            print(f"{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    print("Training complete!")
    return model

In [ ]:
dataloaders = {'train': train_loader, 'test': test_loader}
trained_model = train_model(model, dataloaders, criterion, optimizer, num_epochs=10)

Epoch 1/10
------------------------------


Train Loss: 0.8034 Acc: 0.5037


Test Loss: 0.6994 Acc: 0.5000
Epoch 2/10
------------------------------


Train Loss: 0.6960 Acc: 0.5018


Test Loss: 0.6932 Acc: 0.5000
Epoch 3/10
------------------------------


...

Test Loss: 0.6932 Acc: 0.5000
Epoch 9/10
------------------------------


Train Loss: 0.6939 Acc: 0.5014


Test Loss: 0.6952 Acc: 0.5000
Epoch 10/10
------------------------------


Train Loss: 0.6942 Acc: 0.5039


Test Loss: 0.6931 Acc: 0.5000
Training complete!


In [ ]:
torch.save(trained_model.state_dict(), "/content/drive/My Drive/vgg16_anime_classifier.pth")
print("Model saved!")

Model saved!
